In [2]:
from keras.datasets import cifar10
from keras.utils import to_categorical
import keras

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

首先，我们来定义conv1

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(64, (7,7), padding='valid', strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

为了方便添加残缺模块，我们这里封装成一个方法

In [7]:
def identity_Block(input_data, nb_filter, kernel_size, strides=(1, 1)):
    x = Conv2D(input_data, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2D(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')
    shortcut = Conv2D(inpt, nb_filter=nb_filter, strides=strides, kernel_size=kernel_size)
    x = add([x, shortcut])
    return x

In [ ]:
conv2_x